<td>
   <a target="_blank" href="https://labelbox.com" ><img src="https://labelbox.com/blog/content/images/2021/02/logo-v4.svg" width=256/></a>
</td>


<td>
<a href="https://colab.research.google.com/github/Labelbox/labelbox-notebooks/blob/main/basics/user_management.ipynb" target="_blank"><img
src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>
</td>

<td>
<a href="https://github.com/Labelbox/labelbox-notebooks/tree/main/basics/user_management.ipynb" target="_blank"><img
src="https://img.shields.io/badge/GitHub-100000?logo=github&logoColor=white" alt="GitHub"></a>
</td>

# User Management
* This notebook covers the following:
    * create invites
    * query for remaining allowed invites to an organization
    * set and update organization roles
    * assign users to projects
        * set / update / revoke project role
    * delete users from org

In [ ]:
%pip install "labelbox[data]"

In [ ]:
import labelbox as lb
import os
from labelbox.schema.user_group import UserGroup, UserGroupColor

# API Key and Client
Provide a valid api key below in order to properly connect to the Labelbox Client.

In [ ]:
# Add your api key
API_KEY = None
client = lb.Client(api_key=API_KEY)
organization = client.get_organization()

## Roles
* When inviting a new user to an organization, there are various roles to select from.
* All available roles to your org can be accessed via `client.get_roles()`

In [ ]:
roles = client.get_roles()
for name, role in roles.items():
    print(role.name, ":", role.uid)

* Above we printed out all of the roles available to the current org.
* Notice the `NONE`. That is for project level roles

## Create
* Users are created by sending an invite
* An email will be sent to them and they will be asked to join your organization

### Organization Level Permissions
* Invite a new labeler with labeling permissions on all projects

In [ ]:
# First make sure that you have enough seats:
organization.invite_limit()

In [ ]:
USER_EMAIL = ""
invite = organization.invite_user(USER_EMAIL, roles["LABELER"])

In [ ]:
print(invite.created_at)
print(invite.organization_role_name)
print(invite.email)

### Project Level Permissions
* Invite a new labeler with labeling permissions specific to a set of projects
* Here we set organization level permissions to Roles.NONE to indicate that the user only has project level permissions

In [ ]:
USER_EMAIL = ""
project = client.create_project(name="test_user_management",
                                media_type=lb.MediaType.Image)
project_role = lb.ProjectRole(project=project, role=roles["REVIEWER"])
invite = organization.invite_user(USER_EMAIL,
                                  roles["NONE"],
                                  project_roles=[project_role])

## Read
* Outstanding invites cannot be queried for at this time. This information can be found in the members tab of the web app.
* You are able to query for members once they have joined.

In [ ]:
users = list(organization.users())
print(users[0])

## Update
* There is no update on invites. Instead you must delete and resend them
* You can update User roles

In [ ]:
# Get all users in the organization
users = organization.users()

# Filter the desired user using their email
USER_EMAIL = ""
user = next((u for u in users if u.email == USER_EMAIL), None)

if user:
    print(f"User found: {user.name} ({user.email})")
else:
    print("User not found.")

# Give the user organization level permissions
user.update_org_role(roles["LABELER"])
print(user.org_role())
# Restore project level permissions
user.update_org_role(roles["NONE"])
print(user.org_role())
# Make the user a labeler for the current project
user.upsert_project_role(project, roles["LABELER"])
print(user.org_role())

## Delete
You can remove users from projects and your organization using the SDK. Invites can only be deleted using the **Members** tab on the web platform at this moment.

In [ ]:
# Remove the user from a project
user.remove_from_project(project)
# Alternatively, set the project role to none
user.update_org_role(roles["NONE"])
# Remove the user from the org
organization.remove_user(user)

## Manage user groups
### Create user groups

In [ ]:
# Define a user group
user_group = UserGroup(
    client=client,
    name="New User Group",
    color=UserGroupColor.BLUE
    users=set(user, user1, user2),
    projects=set(project)
)

# Create the defined user group
created_group = user_group.create() 

### Update user groups

In [ ]:
# Define the user group properties to be updated
user_group.name = "Updated User Group Name"
user_group.color = UserGroupColor.GREEN

# Add new projects to the group
projects = []
projects.append(user_group.projects)
projects.append([project_1, project_2])
user_group.projects = projects

# Add new users to the group

users = user_group.users
users.append([new_user_1, new_user_2])
user_group.users = users

# Push the changes to the group
user_group.update()

In [ ]:
## Remove all members and projects from the group
user_group.users = []
user_group.projects = []
user_group.update()

# Push the changes to the group
user_group.update()

In [ ]:
# Delete a user group
user_group.delete()

## Get user group info

In [ ]:
# Get info of a user group
user_group.get()

# Get all user groups in your workspace
user_groups = UserGroup(client).get_user_groups()

# Search for a user group by its name
example_group = next((group for group in user_groups if group.name == "example_name"), None)
if example_group:
    print(f"Found user group 'example_name' with ID: {example_group.id}")
else:
    print("No user group named 'example_name' found")

## Cleanup
Delete the project if you no longer need it:

In [ ]:
project.delete()